In [12]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
from tqdm import tqdm

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
# warnings.filterwarnings('ignore')

In [13]:
import os
import shutil
import tensorflow as tf
RESEARCH_WORK_PATH = "/content/drive/MyDrive/Colab Notebooks/BachelorResearch/"
data_dir = RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/5s_0.5shift/wav/"
if os.path.exists("/content/data/"):
  shutil.rmtree("/content/data/")
shutil.copytree(data_dir + "Q1/", "/content/data/Q1/")
shutil.copytree(data_dir + "Q2/", "/content/data/Q2/")
shutil.copytree(data_dir + "Q3/", "/content/data/Q3/")
shutil.copytree(data_dir + "Q4/", "/content/data/Q4/")
data_dir = "/content/data/"

In [14]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

moods = 'Q1 Q2 Q3 Q4'.split()
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
for g in moods:
    for filename in tqdm(os.listdir(f'/content/data/{g}')):
        songname = f'/content/data/{g}/{filename}'
        y, sr = librosa.load(songname)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        # rmse = librosa.feature.rmse(y=y)
        rmse = librosa.feature.rms(y=y) # librosaのバージョンが0.7.0以上の場合はこちらを実行
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

100%|██████████| 9200/9200 [19:11<00:00,  7.99it/s]


In [15]:
feelings = 'Q1 Q2 Q3 Q4'.split()
csv_path = "/content/data.csv"

data = pd.read_csv(csv_path, usecols=range(0, 28))
data = data[data['label'].isin(feelings)].reset_index(drop=True)
data = data.drop(['filename'],axis=1)
scaler = StandardScaler()
scaler = scaler.fit(np.array(data.iloc[:, :-1], dtype = float))

In [16]:
import pickle
shutil.copy2('/content/data.csv', RESEARCH_WORK_PATH+'note/data5s_0.5shift.csv')
pickle.dump(scaler, open(RESEARCH_WORK_PATH + "/models/1d_scaler_5s_0.5shift.pickle", "wb"))